<a href="https://colab.research.google.com/github/eduardalexandrut/llm-iq-test/blob/main/QwenIQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U git+https://github.com/huggingface/transformers
!pip install PIL
!pip install datasets


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-ulfgmtj3
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ulfgmtj3
  Resolved https://github.com/huggingface/transformers to commit 663c8512398f864cb886879bbb64471777421413
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 42.0 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.47.0.dev0-py3-none-any.whl size=10051213 sha256=3a52ab82e3682f5f47a9406fe3e1a5ddd10eb55babbe2590870382f406174e10
  Stored in directory: /tmp/pip-ephem-wheel-cache-wy8ecja0/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully 

In [2]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor, AutoModelForCausalLM

In [3]:
from PIL import Image
import requests
from datasets import load_dataset

Load `MENSA-visual-iq` dataset from HuggingFace.

In [4]:
dataset = load_dataset("eduardtoni/MENSA-visual-iq-test")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/846 [00:00<?, ?B/s]

(…)-00000-of-00001-25c0febc1da39286.parquet:   0%|          | 0.00/5.12M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/98 [00:00<?, ? examples/s]

Load Qwen2-VL-2B-Instruct model and encoder.

In [5]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype = "auto",
    device_map = "auto",
    trust_remote_code=True
)
processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    trust_remote_code=True
)

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/429M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [29]:
image = dataset['train'][23]['question_img']
possible_sol = dataset['train'][23]['multiple_answer_img']
messages = [
    {'role': 'system', 'content': 'Please reason step by step'},
        {'role': 'user',
        'content': [
            {
                'type': 'image',
            },
            {
                'type':'text',
                'text':'Analyze the previous image: a grid of images with one cell marked with the symbol ?. Identify any visual or logical patterns within rows and columns to understand what the missing cell should contain.'
            },
            {
                'type': 'image',
            },
            {
                'type':'text',
                'text':'Analyze the previous image:This image shows all possible answer choices. Each choice has a unique drawing labeled with a letter above it, which you can select from to complete the missing cell in the first image based on the identified pattern.'
            },
            {
                'type': 'text',
                'text': 'Using the two images, determine the pattern in the first image and select the letter above the drawing in the second image that best completes the missing cell. Return your answer in the following JSON format: {"answer": your selected letter, "explanation": your reasoning here)}. Do not use \"\" symbols within the explanation inside the json.'
            }
        ]
    }
]


text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(
    text = text_prompt,
    images = [image, possible_sol],
    return_tensors = "pt"
)

# Inference: Generation of the output
inputs = inputs.to("cuda")
output_ids = model.generate(**inputs, max_new_tokens = 1024)

generated_text = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(inputs.input_ids, output_ids)
]
output_text = processor.batch_decode(output_ids, skip_special_tokens=True,clean_up_tokenization_spaces=True)

This is a function that given the output of an inference, returns a json object representing the response.

In [10]:
import re
def extract_and_clean_explanation(text):
    # Locate the start of the explanation content
    start = text.find('explanation": "') + len('explanation": "')

    # Find the end quote of the explanation text
    end = text.find('"', start)

    # Extract the content between the quotes
    explanation_content = text[start:end]

    # Remove all double quotes from the explanation content
    cleaned_explanation = explanation_content.replace('"', '')

    return cleaned_explanation

def convert_answer_to_json(answer):
    assistant = '\nassistant\n'
    parts = answer.split(assistant, 1)
    res = parts[1].strip() if len(parts) > 1 else None

    # Remove Markdown-style ```json``` delimiters if present
    cleaned_string = re.sub(r"```json|```", "", res).strip() if res else None

    if cleaned_string:
        try:
            json_object = json.loads(cleaned_string)  # Parse the cleaned string

            # Clean the explanation field
            if "explanation" in json_object:
                json_object["explanation"] = extract_and_clean_explanation(cleaned_string)

            return json_object
        except json.JSONDecodeError as e:
            print(f"JSON decode error: {e}")
            print(res)
            return None
    else:
        return None

In [26]:
import json


# Split and check
print(convert_answer_to_json(output_text[0]))
print(output_text[0])


{'answer': 'B', 'explanation': 'The pattern in the first image is that each row and column contains a different shape. The shapes in the first row are triangles, circles, and squares. The shapes in the second row are diamonds, circles, and squares. The shapes in the third row are triangles, circles, and squares. The shapes in the fourth row are triangles, circles, and squares. The shapes in the fifth row are triangles, circles, and squares. The shapes in the sixth row are triangles, circles, and squares. The shapes in the seventh row are triangles, circles, and squares. The shapes in the eighth row are triangles, circles, and squares. The missing cell in the first image should contain a triangle.'}
system
Please reason step by step
user
Analyze the previous image: a grid of images with one cell marked with the symbol?. Identify any visual or logical patterns within rows and columns to understand what the missing cell should contain.Analyze the previous image:This image shows all possib

We define a function called `test_split()`. This function takes as arguments the start index and the end index of the instnces on which to test the VL-LLM model, the split of the dataset, and the model. The function outputs a `jsonl` stucture containing  the answer letter of each response, the index of the question and the explanation.

In [ ]:
def test_split(split, model, start, end, dataset, prompt):
  dataset_split = dataset
  if split:
    dataset_split = dataset_split[dataset_split['train']]

In [30]:
def test_split(start, end):
  answers_json = []
  for i in range(start,end):
    question_img = dataset['train'][i]['question_img']
    answer_img = dataset['train'][i]['multiple_answer_img']
    text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)

    inputs = processor(
      text = text_prompt,
      images = [question_img, answer_img],
      return_tensors = "pt"
    )

    inputs = inputs.to("cuda")
    output_ids = model.generate(**inputs, max_new_tokens = 512)

    generated_text = [
      output_ids[len(input_ids) :]
      for input_ids, output_ids in zip(inputs.input_ids, output_ids)
    ]
    output_text = processor.batch_decode(output_ids, skip_special_tokens=True,clean_up_tokenization_spaces=True)

    # Convert to JSON
    json_answer = convert_answer_to_json(output_text[0])
    if json_answer:
      json_answer['question_id'] = i
      answers_json.append(json_answer)
    else:
      print(f"No valid JSON found in the generated text from question {i}.")
     # answers_json.append(None)
  return answers_json

In [13]:
answers = test_split(0, 98)
#for answer in answers:
 # print(f'{answer}\n')

KeyboardInterrupt: 

The following function gets the answers given by a LLM and return the score or percentage of correct answers.

In [14]:
def get_score(answers, dataset):
  num_correct_answers = sum(answer['answer'] == dataset['train'][answer['question_id']]['correct_answer'] for answer in answers)
  percentage_correct_answers = (num_correct_answers/len(answers)) * 100
  iq_score = 0
  return (num_correct_answers, percentage_correct_answers, iq_score)

Let's test the score on the whole dataset.

In [ ]:
get_score(answers, dataset)

(20, 20.833333333333336, 0)

Now let's test the model only on MENSA Norway.

In [31]:
mensa_norway_answers = test_split(0,35)
mensa_norway_score = get_score(mensa_norway_answers, dataset)
print(f'Correct Answers: {mensa_norway_score[0]} | Correct Percentage: {mensa_norway_score[1]} | IQ: {mensa_norway_score[2]}' )

Correct Answers: 5 | Correct Percentage: 14.285714285714285 | IQ: 0


Let's write all the answers inside a json file:

In [38]:

with open("mensa-norway-answers.json", "w") as out_file:
  for ans in mensa_norway_answers:
    json_line = json.dumps(ans)
    out_file.write(json_line + "\n")

Now let's test the model only on MENSA Denmark.

In [32]:
mensa_denmark_answers = test_split(35,74)
mensa_denmark_score = get_score(mensa_denmark_answers, dataset)
print(f'Correct Answers: {mensa_denmark_score[0]} | Correct Percentage: {mensa_denmark_score[1]} | IQ: {mensa_denmark_score[2]}' )

Correct Answers: 6 | Correct Percentage: 15.384615384615385 | IQ: 0


In [41]:
with open("mensa-denmark-answers.json", "w") as out_file:
  for ans in mensa_denmark_answers:
    json_line = json.dumps(ans)
    out_file.write(json_line + "\n")

Now let's test the model on the MENSA Sweden.

In [33]:
mensa_sweden_answers = test_split(74,98)
mensa_sweden_score = get_score(mensa_sweden_answers, dataset)
print(f'Correct Answers: {mensa_sweden_score[0]} | Correct Percentage: {mensa_sweden_score[1]} | IQ: {mensa_sweden_score[2]}' )

Correct Answers: 6 | Correct Percentage: 25.0 | IQ: 0


In [42]:
with open("mensa-sweden-answers.json", "w") as out_file:
  for ans in mensa_sweden_answers:
    json_line = json.dumps(ans)
    out_file.write(json_line + "\n")

TO DO:
- Define a proper structure for the answers, by changing functions and saving the obtained values in a json file.**testo in grassetto**
- Modify test_split so that it also takes model and split as arguments(Also prompt strategy?) Don't use json?
- solve dataset problem
- Define function to get percentage of correct answers.
  - How to measure iq score?
- Try different prompts techniques?
- Test Molmo
- Test pix
- Test llama
- llm as a judge to test explanations,

MENSA Norway -> each correct answer has value 1. The range of the score is 85-145, 100 being the average